# Exploratory Data Analysis

The data set is quite strange in how it is tabulated. Each column is a year. Every day a new value for the previous day is added but the earliest value is deleted meaning there is a limit to how many values the TSA publishes to its website simultaneously.

The data is tabulated so that one can compare the number of flights on the same date of the previous year. As such, it is not arranged chronologically. We will have to mess with some date reformatting to get the data to it's desired format.

In [242]:
import pandas as pd

In [243]:
df = pd.read_csv("Data1.csv")

## Formatting and Cleaning Data

Our first goal is to change the data into a time series format, we have to build date-sequences, and then overlay them onto the correct years. 

If you examine the data from the TSA website, you will notice that each year's column begins in the previous year and ends in the current year. Therefore, our data begins from 2018.

In [244]:
df.head()

,Unnamed: 0,Date,2023,2022,2021,2020,2019
0,0,4/10/2023,"2,511,861","2,188,196","1,468,972","102,184","2,484,580"
1,1,4/9/2023,"2,375,705","2,300,909","1,561,495","90,510","2,446,801"
2,2,4/8/2023,"2,093,075","1,959,523","1,378,237","93,645","2,059,142"
3,3,4/7/2023,"2,475,368","2,327,849","1,549,181","108,977","2,590,499"
4,4,4/6/2023,"2,508,487","2,227,151","1,510,829","104,090","2,487,398"


First we create our own date ranges labeled with the correct year since the date column in the original data only contains 2022-2023 dates. We use pandas date ranges in order to do this.

The original data does not contain the leap-day (2020 was a leap year) so we have to remove this day from our 2020 date range.

In [245]:
pd.to_datetime(df["Date"].iloc[0])

Timestamp('2023-04-10 00:00:00')

In [246]:
#Converts a date in the form of a string to 
#date: date in form of string with mm/dd/yyyy
#year: in form of string to a timestamp with the desired year
def format_date(date, year):
    date_list = date.split("/")
    date_list[2] = year
    date = "/".join(date_list)
    date = pd.to_datetime(date)
    return date

In [248]:
def format_to_ts(df, years, truncated_year):
    ts_df = pd.DataFrame({"Date": [], "Checkins": []})
    for year in years:
        #Use our formatting function in a lambda so we can map it on the dataframe
        x = lambda date: format_date(date, year)
        curr_df = df[["Date",year]]
        curr_df["Date"] = curr_df["Date"].map(x)
        curr_df.rename(columns = {year: "Checkins"}, inplace = True)
        ts_df = ts_df.append(curr_df)
    #drops the nan values
    x = lambda date: format_date(date, truncated_year)
    truncated_df = df[["Date", truncated_year]]
    truncated_df.dropna(inplace = True)
    truncated_df["Date"] = truncated_df["Date"].map(x)
    truncated_df.rename(columns = {truncated_year : "Checkins"}, inplace = True)
    ts_df = ts_df.append(truncated_df)
    ts_df = ts_df.sort_values(by = "Date")
    return ts_df

In [249]:
#Formats the comma separated string number into an integer
def to_int(s):
    if s == 'nan':
        return 'nan'
    s = str(s)
    return int(s.replace(",",""))

In [250]:
years = ["2019", "2020", "2021", "2022"]
truncated_year = "2023"

In [251]:
ts_df = format_to_ts(df, years, truncated_year)

<ipython-input-248-db06c96d2c45>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-248-db06c96d2c45>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-248-db06c96d2c45>:9: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-248-db06c96d2c45>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [252]:
ts_df["Checkins"] = ts_df["Checkins"].map(to_int)

In [253]:
ts_df

,Date,Checkins
99,2019-01-01,2202111
98,2019-01-02,2150571
97,2019-01-03,1975947
96,2019-01-04,2229391
95,2019-01-05,2044043
...,...,...
4,2023-04-06,2508487
3,2023-04-07,2475368
2,2023-04-08,2093075
1,2023-04-09,2375705


In [254]:
import plotly.express as pt

In [257]:
pt.line(ts_df, x = "Date", y = "Checkins",title = "TSA Checkins vs Time")

In [258]:
ts_df.to_csv("TSACheckinTS.csv")

## Exploratory Analysis

### Checkins by Month

In [265]:
x = lambda date: date.month
ts_df["Month"] = ts_df["Date"].map(x)

In [268]:
checkins_by_month = ts_df.groupby(["Month"]).mean()

In [285]:
pt.bar(checkins_by_month, checkins_by_month.index,y =  "Checkins", title = "Average Checkins by Month")

### Average Check-ins by Month ex Covid

### Distribution of Checkins

### Checkins Distribution ex Covid

### Autocorrelation of Checkins